In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import explode

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1685163913515_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Carga de datos
headers = ['categoria', 'titulo', 'link']
data = spark.read.option("recursiveFileLookup", "true").csv("s3://parcialnews/headlines/final/").toDF(*headers)

data = data.filter(data["categoria"] != "categoria")

data = data.dropna(subset=["categoria","titulo","link"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
row_count = data.count()
data.show(row_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|           categoria|              titulo|                link|
+--------------------+--------------------+--------------------+
|            Congreso| Reforma de la sa...| eltiempo.com/pol...|
|Conflicto y Narco...| Gobernador del M...| eltiempo.com/jus...|
|            Ciclismo| Rigoberto Urán a...| eltiempo.com/dep...|
|            Medellín| El macabro crime...| eltiempo.com/col...|
|Contenido Patroci...| ¿Por qué su empr...| eltiempo.com/con...|
|       Más Contenido| eCommerce Day Co...| eltiempo.com/mas...|
|Contenido Patroci...| ¿Por qué su empr...| eltiempo.com/con...|
|       Más Contenido| eCommerce Day Co...| eltiempo.com/mas...|
|               Gente| La modelo de Onl...| eltiempo.com/cul...|
|               Gente| Estos son los pa...| eltiempo.com/cul...|
|   Fútbol Colombiano| Los pecados que ...| eltiempo.com/dep...|
|            Medellín| Cayó 'la coja' a...| eltiempo.com/col...|
|              Bogotá| Lo

In [11]:
tokenizer = Tokenizer(inputCol="titulo", outputCol="palabras_titulo")
stopwords_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="titulo_sin_stopwords")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
count_vectorizer = CountVectorizer(inputCol=stopwords_remover.getOutputCol(), outputCol="features")
idf = IDF(inputCol=count_vectorizer.getOutputCol(), outputCol="tfidf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
label_indexer = StringIndexer(inputCol="categoria", outputCol="label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, label_indexer])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
model = pipeline.fit(data_with_filename)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
transformed_data = model.transform(data_with_filename)

transformed_data.write.mode("overwrite").parquet("s3://parcialnews/headlines/final/datos_finales.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
exploded_data = transformed_data.selectExpr("categoria", "titulo", "link")
exploded_data.write.mode("overwrite").csv("s3://parcialnews/headlines/final/final_data.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…